# Kernel Memory plugin for Semantic Kernel

This notebook shows the basic usage of Kernel Memory as a Semantic Kernel Plugin.

First of all, install the Kernel Memory SK Plugin and SK dependencies.

In [ ]:
#r "nuget: Microsoft.KernelMemory.Core, 0.29.240219.2"
#r "nuget: Microsoft.SemanticKernel, 1.5.0"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.29.240219.2"
#r "nuget: Neo4j.Driver, 5.18.1"
#r "../../src/Neo4jMemoryStorage/bin/Debug/net8.0/Neo4j.KernelMemory.MemoryStorage.dll"

using Microsoft.SemanticKernel;
using Microsoft.KernelMemory;
using Neo4j.KernelMemory.MemoryStorage;
using Neo4j.Driver;


## Configuration

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with your OpenAI API Key, and your Memory Service API Key (if you set one).

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> MEMORY_API_KEY=<your KM web service API key>
> ```

In [ ]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

var neo4jConfig = new Neo4jConfig
{
    Uri = env["NEO4J_URI"],
    Username = env["NEO4J_USERNAME"],
    Password = env["NEO4J_PASSWORD"]
};

Console.WriteLine($"Neo4j URI: {neo4jConfig.Uri}");

Let's setup Semantic Kernel as usual:

In [ ]:
var semanticKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: "gpt-3.5-turbo",
        apiKey: env["OPENAI_API_KEY"])
    .Build();

Console.WriteLine("Semantic Kernel ready.");

In [ ]:
var kernelMemory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .WithNeo4j(neo4jConfig)
    .Build<MemoryServerless>();

## Import Memory Plugin into SK

Wrap the KernelMemory in a MemoryPlugin, and add it to the SK instance.

In [ ]:

// Name of the plugin. This is the name you'll use in skPrompt, e.g. {{memory.ask ...}}
var pluginName = "memory";

// Import the plugin into the kernel.
// 'waitForIngestionToComplete' set to true forces memory write operations to wait for completion.
var memoryPlugin = semanticKernel.ImportPluginFromObject(
    new MemoryPlugin(kernelMemory, waitForIngestionToComplete: true),
    pluginName);

Console.WriteLine("Memory plugin imported.");

## Populate memory with sample information

Let's store some data in memory:

1. storing some text, e.g. Wikipedia's description of Orion
2. and importing a PDF document, e.g. a news from NASA about Orion spacecraft

To import data you can use either the web client connector or the plugin, here we use both just for demo purposes.

When working with SK Planners and Semantic Functions (like the one below) you should always
use KM Plugin. You can also use the web client connector to create new plugins and customize
how you interact with your memories, e.g. selecting indexes, using filters, customizing chunking, etc.

In [ ]:

// Save a string using kernel memory
await kernelMemory.ImportTextAsync(
    "Orion is a prominent set of stars visible during winter in " +
    "the northern celestial hemisphere. It is one of the 88 modern constellations; " +
    "it was among the 48 constellations listed by the 2nd-century astronomer Ptolemy. " +
    "It is named for a hunter in Greek mythology.", 
    documentId: "OrionDefinition");

// Save a PDF file using the plugin
var context = new KernelArguments
{
    [MemoryPlugin.FilePathParam] = "NASA-news.pdf",
    [MemoryPlugin.DocumentIdParam] = "NASA001"
};
await memoryPlugin["SaveFile"].InvokeAsync(semanticKernel, context);

// Save a string with unique, fictional information that is not publicly available
await kernelMemory.ImportTextAsync(
    "Deer Horatio is a real person, though he is often confused with a character from Shakespeare's Hamlet. " +
    "He lives near the pier in a hamlet called Shake. " +
    "He is well aware that AI dreams of electric sheep.", 
    documentId: "DeerHoratioBio");

Console.WriteLine("Memory updated.");

## Use Memory in a semantic function

Here we define a simple semantic function, using `{{memory.ask}}` to
fetch information from memory.

In [ ]:
var skPrompt = """
Question to Memory: {{$input}}

Answer from Memory: {{memory.ask $input}}

If the answer is empty say 'I don't know' otherwise reply with a preview of the answer,
truncated to 15 words. Prefix with one emoji relevant to the content.
""";

var myFunction = semanticKernel.CreateFunctionFromPrompt(skPrompt);

Console.WriteLine("Semantic Function ready.");

Let's now interact with our memories using `myFunction` function, asking questions, leveraging KM and LLMs to generate answers.

Our function internally uses KM Plugin to retrieve an answer, and then custom some semantic logic to decide how to format the output.

In [ ]:
var answer = await myFunction.InvokeAsync(semanticKernel,
    "any news from NASA about Orion?");

Console.WriteLine(answer);

In [ ]:
var answer = await myFunction.InvokeAsync(semanticKernel,
    "usage of the word 'Orion'");

Console.WriteLine(answer);

In [ ]:
var answer = await myFunction.InvokeAsync(semanticKernel,
    "Where does Horatio live?");

Console.WriteLine(answer);

In [ ]:
var answer = await myFunction.InvokeAsync(semanticKernel,
    "What is Horatio's full name?");

Console.WriteLine(answer);

In [ ]:
var answer = await myFunction.InvokeAsync(semanticKernel,
    "Tell me about Horatio.");

Console.WriteLine(answer);